In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
!mkdir -p save/model/formatted_lines_sitcom/1-1_512

In [ ]:
import boto3
bucket_name = 'chandlerbotmodels'

s3_file_path= 'model/4000_backup_bidir_model.tar'
save_as = 'save/model/formatted_lines_sitcom/1-1_512/4000_backup_bidir_model.tar'


s3 = boto3.client(
    's3',
    aws_access_key_id='',
aws_secret_access_key=''
    #aws_session_token=SESSION_TOKEN,
)

s3.download_file(bucket_name , s3_file_path, save_as)

In [ ]:
!wget -c 'https://s3.amazonaws.com/chandlerbotmodels/model/4000_backup_bidir_model.tar'

In [ ]:
!ls

In [ ]:
#!rm save/model/formatted_lines_sitcom/1-1_512/2000_round2_bidir_model.tar

In [ ]:
!mv 2000_round2_bidir_model.tar save/model/formatted_lines_sitcom/1-1_512/2000_round2_bidir_model.tar

In [ ]:
!ls

In [ ]:
!mkdir -p save/training_data/formatted_lines_sitcom

In [ ]:

s3_file_path= 'training_data/4000_training_batches_64.tar'
save_as = 'save/training_data/formatted_lines_sitcom/4000_training_batches_64.tar'
s3.download_file(bucket_name , s3_file_path, save_as)
s3_file_path= 'training_data/pairs.tar'
save_as = 'save/training_data/formatted_lines_sitcom/pairs.tar'
s3.download_file(bucket_name , s3_file_path, save_as)
s3_file_path= 'training_data/voc.tar'
save_as = 'save/training_data/formatted_lines_sitcom/voc.tar'
s3.download_file(bucket_name , s3_file_path, save_as)

In [ ]:
!ls save/training_data/formatted_lines_sitcom

In [ ]:
!wget https://www.dropbox.com/s/9827g9ytuqqibb3/4000_backup_bidir_model.tar?dl=0

In [ ]:
!mv 4000_backup_bidir_model.tar?dl=0 'save/model/formatted_lines_sitcom/1-1_512/4000_backup_bidir_model.tar'

In [ ]:
!ls 'save/model/formatted_lines_sitcom/1-1_512'

In [ ]:
import torch

In [ ]:
torch.__version__

In [ ]:
!mkdir data

In [ ]:
! ls "../input"

In [ ]:
!mv "../input/formatted_lines_sitcom.txt" data

In [ ]:
!ls data

In [ ]:
!git clone https://github.com/ywk991112/pytorch-chatbot

In [ ]:
!ls

In [ ]:
!mv "4000_backup_bidir_model.tar?dl=0"  "4000_backup_bidir_model.tar"

In [ ]:
!mkdir -p save/model/formatted_lines_sitcom/1-1_512

In [ ]:
!mv 4000_backup_bidir_model.tar save/model/formatted_lines_sitcom/1-1_512

In [ ]:
!cd pytorch-chatbot

In [ ]:
!ls save/training_data/formatted_lines_sitcom

In [ ]:
!wget https://www.dropbox.com/s/1iwo8pk8f1dgbnm/pairs.tar?dl=0

In [ ]:
!wget https://www.dropbox.com/s/isa1uvw2n1yhl2x/voc.tar?dl=0

In [ ]:
!ls save

In [ ]:
!mkdir save/training_data

In [ ]:
!mkdir -p save/training_data/formatted_lines_sitcom

In [ ]:
!mv voc.tar?dl=0 voc.tar

In [ ]:
!mv voc.tar save/training_data/formatted_lines_sitcom 

In [ ]:
! ls pytorch-chatbot

In [ ]:
!wget https://www.dropbox.com/s/fqgxv47ws3syt51/load_round2.py?dl=0

In [ ]:
!mv load_round2.py?dl=0 load.py

In [ ]:
!rm pytorch-chatbot/load.py 

In [ ]:
!ls pytorch-chatbot

In [ ]:
!mv load.py pytorch-chatbot 

In [ ]:
!ls pytorch-chatbot 

In [ ]:
#!python pytorch-chatbot/main.py -tr data/formatted_lines_sitcom.txt -l save/model/formatted_lines_sitcom/1-1_512/4000_backup_bidir_model.tar -lr 0.0001 -it 4000 -b 64 -p 500 -s 1000

In [ ]:
import sys, os
sys.path.append("pytorch-chatbot/")
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch.backends.cudnn as cudnn

import itertools
import random
import math
import os
from tqdm import tqdm
from load import loadPrepareData
from load import SOS_token, EOS_token, PAD_token
from model import EncoderRNN, LuongAttnDecoderRNN
from config import MAX_LENGTH, teacher_forcing_ratio, save_dir

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

cudnn.benchmark = True
#############################################
# generate file name for saving parameters
#############################################
def filename(reverse, obj):
	filename = ''
	if reverse:
		filename += 'reverse_'
	filename += obj
	return filename


#############################################
# Prepare Training Data
#############################################
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

# batch_first: true -> false, i.e. shape: seq_len * batch
def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# convert to index, add EOS
# return input pack_padded_sequence
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = [len(indexes) for indexes in indexes_batch]
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# convert to index, add EOS, zero padding
# return output variable, mask, max length of the sentences in batch
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# pair_batch is a list of (input, output) with length batch_size
# sort list of (input, output) pairs by input length, reverse input
# return input, lengths for pack_padded_sequence, output_variable, mask
def batch2TrainData(voc, pair_batch, reverse):
    if reverse:
        pair_batch = [pair[::-1] for pair in pair_batch]
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

#############################################
# Training
#############################################

def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, max_length=MAX_LENGTH):

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    loss = 0
    print_losses = []
    n_totals = 0

    encoder_outputs, encoder_hidden = encoder(input_variable, lengths, None)

    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    decoder_hidden = encoder_hidden[:decoder.n_layers]

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Run through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden, _ = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_input = target_variable[t].view(1, -1) # Next input is current target
            loss += F.cross_entropy(decoder_output, target_variable[t], ignore_index=EOS_token)
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden, decoder_attn = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            _, topi = decoder_output.topk(1) # [64, 1]

            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            loss += F.cross_entropy(decoder_output, target_variable[t], ignore_index=EOS_token)

    loss.backward()

    clip = 50.0
    _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / max_target_len 


def trainIters(corpus, reverse, n_iteration, learning_rate, batch_size, n_layers, hidden_size,
                print_every, save_every, dropout, loadFilename=None, attn_model='dot', decoder_learning_ratio=5.0):

    voc, pairs = loadPrepareData(corpus)

    # training data
    corpus_name = os.path.split(corpus)[-1].split('.')[0]
    training_batches = None
    try:
        training_batches = torch.load(os.path.join(save_dir, 'training_data', corpus_name,
                                                   '{}_{}_{}.tar'.format(n_iteration, \
                                                                         filename(reverse, 'training_batches'), \
                                                                         batch_size)))
    except FileNotFoundError:
        print('Training pairs not found, generating ...')
        training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)], reverse)
                          for _ in range(n_iteration)]
        torch.save(training_batches, os.path.join(save_dir, 'training_data', corpus_name,
                                                  '{}_{}_{}.tar'.format(n_iteration, \
                                                                        filename(reverse, 'training_batches'), \
                                                                        batch_size)))
    # model
    checkpoint = None
    print('Building encoder and decoder ...')
    embedding = nn.Embedding(voc.n_words, hidden_size)
    encoder = EncoderRNN(voc.n_words, hidden_size, embedding, n_layers, dropout)
    attn_model = 'dot'
    decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.n_words, n_layers, dropout)
    if loadFilename:
        checkpoint = torch.load(loadFilename)
        encoder.load_state_dict(checkpoint['en'])
        decoder.load_state_dict(checkpoint['de'])
    # use cuda
    encoder = encoder.to(device)
    decoder = decoder.to(device)

    # optimizer
    print('Building optimizers ...')
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
    if loadFilename:
        encoder_optimizer.load_state_dict(checkpoint['en_opt'])
        decoder_optimizer.load_state_dict(checkpoint['de_opt'])

    # initialize
    print('Initializing ...')
    start_iteration = 1
    perplexity = []
    print_loss = 0
    if loadFilename:
        #start_iteration = checkpoint['iteration'] + 1
        start_iteration = 0
        perplexity = checkpoint['plt']
    try:
	    for iteration in tqdm(range(start_iteration, n_iteration + 1)):
	        training_batch = training_batches[iteration - 1]
	        input_variable, lengths, target_variable, mask, max_target_len = training_batch

	        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
	                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size)
	        print_loss += loss
	        perplexity.append(loss)

	        if iteration % print_every == 0:
	            print_loss_avg = math.exp(print_loss / print_every)
	            print('%d %d%% %.4f' % (iteration, iteration / n_iteration * 100, print_loss_avg))
	            print_loss = 0

	        if (iteration % save_every == 0):
	            directory = os.path.join(save_dir, 'model', corpus_name, '{}-{}_{}'.format(n_layers, n_layers, hidden_size))
	            if not os.path.exists(directory):
	                os.makedirs(directory)
	            torch.save({
	                'iteration': iteration,
	                'en': encoder.state_dict(),
	                'de': decoder.state_dict(),
	                'en_opt': encoder_optimizer.state_dict(),
	                'de_opt': decoder_optimizer.state_dict(),
	                'loss': loss,
	                'plt': perplexity
	            }, os.path.join(directory, '{}_{}.tar'.format(iteration, filename(reverse, 'round2_bidir_model'))))
    except Exception as e:  print(e)

In [ ]:
def parseFilename(filename, test=False):
    filename = filename.split('/')
    dataType = filename[-1][:-4] # remove '.tar'
    parse = dataType.split('_')
    reverse = 'reverse' in parse
    layers, hidden = filename[-2].split('_')
    n_layers = int(layers.split('-')[0])
    hidden_size = int(hidden)
    return n_layers, hidden_size, reverse
n_layers, hidden_size, reverse = parseFilename('save/model/formatted_lines_sitcom/1-1_512/4000_backup_bidir_model.tar')
print(n_layers, hidden_size, reverse)

In [ ]:
loadFilename='save/model/formatted_lines_sitcom/1-1_512/4000_backup_bidir_model.tar'
n_iteration=1500
learning_rate=0.0001
batch_size=64
print_every=500
save_every=1500
dropout=0.1
trainIters('data/formatted_lines_sitcom.txt', reverse, n_iteration, learning_rate, batch_size,
                    n_layers, hidden_size, print_every, save_every, dropout, loadFilename=loadFilename)

In [ ]:
#!python pytorch-chatbot/main.py -tr data/formatted_lines_sitcom.txt -l save/model/formatted_lines_sitcom/1-1_512/4000_backup_bidir_model.tar -lr 0.0001 -it 4000 -b 64 -p 500 -s 1000

In [ ]:
!ls data

In [ ]:
!ls save/model/formatted_lines_sitcom

In [ ]:
!ls save/model/formatted_lines_sitcom/1-1_512

In [ ]:
!pip install boto3

In [ ]:
!ls save/training_data/formatted_lines_sitcom

In [ ]:
import boto3
bucket_name = 'chandlerbotmodels'

content = open('save/training_data/formatted_lines_sitcom/voc.tar', 'rb')
s3 = boto3.client('s3')
s3 = boto3.client(
    's3',
    aws_access_key_id='',
    aws_secret_access_key='',
    #aws_session_token=SESSION_TOKEN,
)
s3.put_object(
   Bucket=bucket_name, 
   Key='training_data/voc.tar', 
   Body=content
)